## Simple RNN

In [13]:
import torch
import torch.nn as nn
import torch.nn as F
import numpy as np

In [59]:
sample = 'if you want this, you should go to the mart'
char_set = list(set(sample))
# vocab
char_dic = {c: i for i, c in enumerate(char_set)}

sample_idx = [char_dic[c] for c in sample]

x_data = [sample_idx[:-1]]
x_oh = [np.eye(len(char_dic))[x] for x in x_data]
y_data = [sample_idx[1:]]

X = torch.FloatTensor(x_oh)
y = torch.LongTensor(y_data)

In [81]:
# nn.RNN(input_dim, units, batch_first=True)
rnn = nn.RNN(len(char_dic), 15, batch_first=True)
criterion = nn.CrossEntropyLoss()
optm = torch.optim.Adam(rnn.parameters(), lr=1e-2)

In [43]:
# X가 포함하고 있는 정보 : (N, X의 timestep, input_dim)
# rnn(X)[0].shape : (N, X의 timestep, units) # 출력 텐서
# rnn(X)[1].shape : (N, last timestep, units) # last hidden state
hidden_states, last_hidden_state = rnn(X)

# 참고자료 : https://stackoverflow.com/questions/48302810/whats-the-difference-between-hidden-and-output-in-pytorch-lstm

In [98]:
# W_x : 뉴런개수 * input_dim
# W_h : 뉴런개수 * 뉴런개수
# b_x : 뉴런개수 
# b_h : 뉴런개수 (위와 값이 다름)
list(rnn.named_parameters())[2:4]

[('bias_ih_l0',
  Parameter containing:
  tensor([ 0.1206,  0.1548,  0.0532, -0.0993, -0.1528,  0.0448, -0.2576, -0.0419,
          -0.2149,  0.2073, -0.1898,  0.0155, -0.1937, -0.1987,  0.2428],
         requires_grad=True)),
 ('bias_hh_l0',
  Parameter containing:
  tensor([ 0.2065,  0.0845,  0.0586, -0.0389, -0.1854,  0.1904, -0.0725,  0.1603,
           0.0461, -0.1152, -0.1681, -0.0425, -0.1990, -0.0569,  0.2473],
         requires_grad=True))]

In [53]:
for i in range(50):
    optm.zero_grad()
    output, last_hidden_state = rnn(X)
    # 배치 제거
    loss = criterion(output.view(-1, len(char_dic)), y.view(-1))
    loss.backward()
    optm.step()
    
    # 가장 높은 feature의 idx
    y_hat = output.data.numpy().argmax(axis=2)
    # y_hat이 2차원이니까 squeeze로 쓸모 없는 [] 삭제해줘야함
    y_hat_str = ''.join([char_set[c] for c in np.squeeze(y_hat)])
    
    if i % 5 == 0:
        print(y_hat_str)
    

f you thnt  his, you thouldhgo     he maut
f you thnt  his, you thouldhgo     he maut
f you thnt  his, you thouldhgo    the maut
f you tant  his, you thouldhgo to the maut
f you tant  his, you thould go to the mart
f you tant  his, you thould go to the mort
f you tant this, you thould go to the mart
f you tant this, you thould go to the mart
f you tant this, you thould go to the mart
f you tant this, you thould go to the mart


In [109]:
# LSTM은 어떤 식으로 output이 구성되어 있을까? 

input_dim, units = len(char_dic), 15
lstm = nn.LSTM(input_dim, units, batch_first=True)

hidden_states, last_states = lstm(X)
last_hidden_state, last_cell_state = last_states

IndexError: list index out of range